In [1]:
import pandas as pd
from tqdm.notebook import tqdm

subtypes_from_TCGAbiolinks.csv was downloaded using the TCGAbiolinks package no 2021-07-28 and saved to a CSV file. More info on TCGAbiolinks:

Here are the shell script commands used in a terminal on the GenePattern Notebook server:
```
source activate python3.8
conda install -c bioconda bioconductor-tcgabiolinks=2.18.0
R
#Now we are running inside of R
library(TCGAbiolinks)
subtypes <- PanCancerAtlas_subtypes()
write.csv(subtypes,'/home/jovyan/subtypes_from_TCGAbiolinks.csv')
q()
source deactivate python3.8
```


https://www.bioconductor.org/packages/release/bioc/vignettes/TCGAbiolinks/inst/doc/subtypes.html#PanCancerAtlas_subtypes:_Curated_molecular_subtypes

In [3]:
subtypes = pd.read_csv('subtypes_from_TCGAbiolinks.csv',index_col=1)
print('subtypes.shape=',subtypes.shape)
# subtypes.head()

subtypes.shape= (7734, 10)


In [13]:
# subtypes.tail()

In [6]:
new_index = subtypes.index.to_frame().apply(lambda x: x['pan.samplesID'][:12],axis=1)

In [7]:
p_n = subtypes.index.to_frame().apply(lambda x: x['pan.samplesID'][13:15],axis=1)

In [8]:
len(new_index.unique())

7606

In [9]:
subset = subtypes[['Subtype_Selected']]
subset['TCGA_ID'] = new_index.copy()
subset['Primary/Normal'] = p_n.copy()

<ipython-input-9-331e6b47cfc0>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset['TCGA_ID'] = new_index.copy()
<ipython-input-9-331e6b47cfc0>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset['Primary/Normal'] = p_n.copy()


In [11]:
# Removing all Normal samples
print(f"{len(subset[subset['Primary/Normal']=='11'])} Normals")
subset = subset[subset['Primary/Normal']!='11']
print(f"{len(subset[subset['Primary/Normal']=='06'])} Metastasis")
subset = subset[subset['Primary/Normal']!='06']
print(f"{len(subset[subset['Primary/Normal']=='05'])} New Primary")
subset = subset[subset['Primary/Normal']!='05']
# duplicated = subset[subset['TCGA_ID'].duplicated(False)]['TCGA_ID'].unique()

114 Normals
275 Metastasis
3 New Primary


In [12]:
subset.shape

(7342, 3)

In [13]:
len(subset['TCGA_ID'].unique())

7342

In [21]:
subset['Cancer_Type'] = subset['Subtype_Selected'].apply(lambda x: x.split('.')[0])
subset['Cancer_Subtype'] = subset['Subtype_Selected'].apply(lambda x: x.split('.')[1])
subset

,Subtype_Selected,TCGA_ID,Primary/Normal,Cancer_Type,Cancer_Subtype
pan.samplesID,,,,,
TCGA-OR-A5J1,ACC.CIMP-high,TCGA-OR-A5J1,,ACC,CIMP-high
TCGA-OR-A5J2,ACC.CIMP-low,TCGA-OR-A5J2,,ACC,CIMP-low
TCGA-OR-A5J3,ACC.CIMP-intermediate,TCGA-OR-A5J3,,ACC,CIMP-intermediate
TCGA-OR-A5J4,ACC.CIMP-high,TCGA-OR-A5J4,,ACC,CIMP-high
TCGA-OR-A5J5,ACC.CIMP-intermediate,TCGA-OR-A5J5,,ACC,CIMP-intermediate
...,...,...,...,...,...
TCGA-ND-A4WA-01A-12R-A28V-07,UCS.2,TCGA-ND-A4WA,01,UCS,2
TCGA-NG-A4VU-01A-11R-A28V-07,UCS.1,TCGA-NG-A4VU,01,UCS,1
TCGA-QN-A5NN-01A-11R-A28V-07,UCS.2,TCGA-QN-A5NN,01,UCS,2


In [23]:
subset[['TCGA_ID','Cancer_Type','Cancer_Subtype']].to_excel('subtypes_from_TCGAbiolinks_preprocessed.xlsx',index=False)